**Install python dependencies**

In [1]:
!pip install -q -r ./requirements.txt

**Load python libraries**

In [2]:
from sqlalchemy import create_engine
from os import getenv
import pandas as pd

**Connect to database**

In [3]:
db_name = getenv("POSTGRES_DB")
db_user = getenv("POSTGRES_USER")
db_pass = getenv("POSTGRES_PASSWORD")
db_host = 'localhost'
db_port = '5432'
db_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_user, db_pass, db_host, db_port, db_name)
db = create_engine(db_string)

**Preprocess Data: Event Data**

In [4]:
# getting event data from database
db_response = db.execute(
    f"""select  (coalesce(p.first_name, '') || ' ' ||coalesce(p.last_name, '')) as name,
       position,
       number as matchday,
       type as event_type,
       count(*) from events
        inner join matchdays m on events.matchday_id = m.id
        inner join players p on events.player_id = p.id
    where corrected is false
    group by name, position, matchday, event_type
    order by name, matchday,  event_type
    """
)

df_event = pd.DataFrame(db_response.fetchall())
df_event.columns = db_response.keys()

df_event

,name,position,matchday,event_type,count
0,Aarón,defender,3,interception,1
1,Aarón,defender,3,pass,10
2,Aarón,defender,3,playerOn,1
3,Aarón,defender,3,successfulTackle,1
4,Aarón,defender,3,throwIn,2
...,...,...,...,...,...
72211,Yuya Osako,attacker,34,shotAtGoal,1
72212,Yuya Osako,attacker,34,successfulTackle,2
72213,Yuya Osako,attacker,34,superPass,1
72214,Yuya Osako,attacker,34,unsuccessfulPass,3


In [5]:
# load event_types
event_types = ['pass', 'unsuccessfulPass', 'superPass', 'throwIn', 'goalAssist', 'farCorner', 'cross', 'freeKick', 'interception', 'goalMissedFar', 'shotAtGoal', 'unsuccessfulPenalty', 'missedChance', 'goal', 'doublePack', 'hattrick', 'penaltyGoal', 'ownGoal', 'foul','awardedPenalty', 'causedPenalty', 'yellowCard', 'secondYellowCard', 'redCard', 'successfulTackle', 'unsuccessfulTackle', 'blockedGoalShot', 'offside', 'error', 'savedPenalty', 'lostPenalty', 'defended', 'goalAgainst',]

In [16]:
df_players = df_event.groupby('name')

print(33*34) # 1122

for player_tuple in df_players:
    player_name = player_tuple[0]
    position = player_tuple[1]['position'].max()

    df_player = player_tuple[1]

    for matchday in range(1,35):
        # if matchday does not exist
        if df_player.loc[df_player['matchday'] == matchday].empty == True:
            # add for every event type 0
            for event_type in event_types:
                df_player = df_player.append({
                    'name': player_name, 
                    'position': position,
                    'matchday': matchday,
                    'event_type': event_type,
                    'count': 0
                }, ignore_index=True)

        # if match does exist
        if df_player.loc[df_player['matchday'] == matchday].empty == False:
            for event_type in event_types:
             # if no entry for event exists
              if df_player.loc[df_player['matchday'] == matchday].empty == True: 
                  # add 0
                  df_player = df_player.append({
                    'name': player_name, 
                    'position': position,
                    'matchday': matchday,
                    'event_type': event_type,
                    'count': 0
                }, ignore_index=True)
    
    print(df_player.shape[0])

    




1122
999
694
649
587


KeyboardInterrupt: 

In [19]:
df_players = df_event.groupby('name')

for player_tuple in df_players:
    player_name = player_tuple[0]
    position = player_tuple[1]['position'].max()

    df_player = player_tuple[1]

    for matchday in range(1,35):
        for event_type in event_types:
            if df_player.loc[(df_player['matchday'] == matchday) & (df_player['event_type'] == event_type)].empty == True:
                df_player = df_player.append({
                    'name': player_name, 
                    'position': position,
                    'matchday': matchday,
                    'event_type': event_type,
                    'count': 0
                }, ignore_index=True)
    
    print(player_name)
    print(df_player.shape[0])


1122
 Aarón
1
pass
 Aarón
1
unsuccessfulPass
 Aarón
1
superPass
 Aarón
1
throwIn
 Aarón
1
goalAssist
 Aarón
1
farCorner
 Aarón
1
cross
 Aarón
1
freeKick
 Aarón
1
interception
 Aarón
1
goalMissedFar
 Aarón
1
shotAtGoal
 Aarón
1
unsuccessfulPenalty
 Aarón
1
missedChance
 Aarón
1
goal
 Aarón
1
doublePack
 Aarón
1
hattrick
 Aarón
1
penaltyGoal
 Aarón
1
ownGoal
 Aarón
1
foul
 Aarón
1
awardedPenalty
 Aarón
1
causedPenalty
 Aarón
1
yellowCard
 Aarón
1
secondYellowCard
 Aarón
1
redCard
 Aarón
1
successfulTackle
 Aarón
1
unsuccessfulTackle
 Aarón
1
blockedGoalShot
 Aarón
1
offside
 Aarón
1
error
 Aarón
1
savedPenalty
 Aarón
1
lostPenalty
 Aarón
1
defended
 Aarón
1
goalAgainst
 Aarón
2
pass
 Aarón
2
unsuccessfulPass
 Aarón
2
superPass
 Aarón
2
throwIn
 Aarón
2
goalAssist
 Aarón
2
farCorner
 Aarón
2
cross
 Aarón
2
freeKick
 Aarón
2
interception
 Aarón
2
goalMissedFar
 Aarón
2
shotAtGoal
 Aarón
2
unsuccessfulPenalty
 Aarón
2
missedChance
 Aarón
2
goal
 Aarón
2
doublePack
 Aarón
2
hattrick
 Aarón
2

KeyboardInterrupt: 